IMPORTS

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import keras 
from keras.models import Sequential 
from keras.layers import Dense
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

UNZIPPING THE DATASET FILE

In [ ]:
!cd "/content/"
!ls
!unzip 'a01c26dcd27711ea.zip'

a01c26dcd27711ea.zip  sample_data
Archive:  a01c26dcd27711ea.zip
   creating: Dataset/
  inflating: Dataset/train.csv       
  inflating: Dataset/test.csv        


LOADING TRAIN AND TEST DATASET

In [ ]:
train_df = pd.read_csv("/content/Dataset/train.csv")
test_df = pd.read_csv("/content/Dataset/test.csv")
train_df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


checking for nan

In [ ]:
print(train_df.shape)
test_df.isnull().sum(axis=0)

(18834, 11)


pet_id            0
issue_date        0
listing_date      0
condition       619
color_type        0
length(m)         0
height(cm)        0
X1                0
X2                0
dtype: int64

FEATURE ENGINEERING

In [ ]:
data = [train_df,test_df]
for dataset in data:
  dataset['issue_year'] = pd.DatetimeIndex(dataset['issue_date']).year
  dataset['listing_year'] = pd.DatetimeIndex(dataset['listing_date']).year
  dataset['date_diff'] = (pd.DatetimeIndex(dataset['listing_date'])-pd.DatetimeIndex(dataset['issue_date']))//np.timedelta64(1,'D')
  dataset.issue_year.loc[dataset.issue_year<2005]=2005
  dataset['days_bin'] = pd.qcut(dataset.date_diff,5,labels=[1,2,3,4,5]).astype(int)
  dataset['length_bin'] = pd.qcut(dataset['length(m)'],5,labels=[1,2,3,4,5]).astype(int)
  dataset['height_bin'] = pd.qcut(dataset['height(cm)'],5,labels=[1,2,3,4,5]).astype(int)
train_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category,issue_year,listing_year,date_diff,days_bin,length_bin,height_bin
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1,2016,2016,73,1,4,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2,2013,2018,1862,5,4,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4,2014,2016,752,4,1,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2,2016,2019,755,4,4,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1,2017,2017,52,1,3,1


CHECKING RELATION TO FILL NAs

In [ ]:
# fig, ax = plt.subplots(figsize=(15,7))
test_df.issue_year.loc[test_df.issue_year<2005]=2005
train_df.issue_year.loc[train_df.issue_year<2005]=2005
# train_df['X1_bin'] = pd.qcut(train_df.X1,5,labels=[1,2,3,4,5]).astype(int)
train_df[['condition', 'X2']].groupby(['condition']).mean()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,X2
condition,
0.0,3.424455
1.0,3.723127
2.0,7.201081


FILLING NA USING RANDOM FOREST CLASSIFIER

In [ ]:
data = [train_df,test_df]
for dataset in data:
  df_sub = dataset[['X1','X2','issue_year','listing_year','days_bin','length_bin','height_bin','condition']]

  X_train  = df_sub.dropna().drop('condition', axis=1)
  y_train  = dataset['condition'].dropna()
  X_test = df_sub.loc[np.isnan(dataset.condition)].drop('condition', axis=1)

  regressor = RandomForestClassifier(n_estimators = 300)
  regressor.fit(X_train, y_train)
  y_pred = np.round(regressor.predict(X_test),1)
  dataset['condition'].loc[dataset['condition'].isnull()] = y_pred

train_df['condition'].isnull().sum(axis=0) # no more NAN now
test_df['condition'].isnull().sum(axis=0)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0

In [ ]:
test_df['condition'].isnull().sum(axis=0)
train_df['pet_category'].nunique()

4

CREATING DUMMIES TO MAKE THE DATA BETTER FOR LINEAR MODEL

In [ ]:
df = pd.concat([train_df, pd.get_dummies(train_df['color_type'])], axis=1)
test = pd.concat([test_df, pd.get_dummies(test_df['color_type'])], axis=1)
arr = train_df['X1'].unique()
a = train_df['X2'].unique()
arr1 = test_df['X1'].unique()
a1 = test_df['X2'].unique()
print(np.delete(arr, arr1))
l = pd.get_dummies(train_df['X1'])
m = pd.get_dummies(train_df['X2'])
l1 = pd.get_dummies(test_df['X1'])
m1 = pd.get_dummies(test_df['X2'])
b = []
c=[]
b1=[]
c1=[]
for i in arr:
  b.append("X1"+str(i))
for i in a:
  c.append("X2"+str(i))
for i in arr1:
  b1.append("X1"+str(i))
for i in a1:
  c1.append("X2"+str(i))
res = dict(zip(arr, b))
resu = dict(zip(a, c))
res1 = dict(zip(arr1, b1))
resu1 = dict(zip(a1, c1))
l.rename(res, errors="raise",axis=1, inplace = True)
m.rename(resu, errors="raise",axis=1, inplace = True)
l1.rename(res1, errors="raise",axis=1, inplace = True)
m1.rename(resu1, errors="raise",axis=1, inplace = True)
df = pd.concat([df,l], axis=1)
df = pd.concat([df,m], axis=1)
test = pd.concat([test,l1], axis=1)
test = pd.concat([test,m1], axis=1)
test.insert(test.columns.get_loc('X118')+1, "X119",0)
test.insert(df.columns.get_loc('X12')+1, "X13",0)
test.head()

[18 14]


,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,issue_year,listing_year,date_diff,days_bin,length_bin,height_bin,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,...,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle,X10,X11,X12,X14,X15,X16,X17,X13,X18,X19,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7,2005,2017,4404,5,5,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1,2018,2019,174,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7,2012,2018,1999,5,2,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1,2015,2018,1148,4,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7,2017,2018,463,3,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


ADDING MISSING COLUMNS AND FINISHING FEATURE ENGINEERING

In [ ]:
X = df.drop(labels=['pet_id','issue_date','listing_date','color_type','length(m)','height(cm)','breed_category','pet_category'],axis=1)
t = test.drop(labels=['pet_id','issue_date','listing_date','color_type','length(m)','height(cm)'],axis=1)
t.insert(15, "Black Tiger",0)
t.insert(28, "Brown Tiger",0)
y1 = df['breed_category']
y2 = df['pet_category']

X.head()

,condition,X1,X2,issue_year,listing_year,date_diff,days_bin,length_bin,height_bin,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Black Tiger,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Brown Tiger,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,...,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29
0,2.0,13,9,2016,2016,73,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1.0,13,9,2013,2018,1862,5,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1.0,15,4,2014,2016,752,4,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,1.0,0,1,2016,2019,755,4,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,2.0,18,4,2017,2017,52,1,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
t.head()

,condition,X1,X2,issue_year,listing_year,date_diff,days_bin,length_bin,height_bin,Agouti,Apricot,Black,Black Brindle,Black Smoke,Black Tabby,Black Tiger,Blue,Blue Cream,Blue Merle,Blue Point,Blue Smoke,Blue Tabby,Blue Tick,Blue Tiger,Brown,Brown Brindle,Brown Merle,Brown Tabby,Brown Tiger,Buff,Calico,Calico Point,Chocolate,Chocolate Point,Cream,Cream Tabby,Fawn,Flame Point,Gold,Gray,...,Silver Lynx Point,Silver Tabby,Tan,Torbie,Tortie,Tortie Point,Tricolor,White,Yellow,Yellow Brindle,X10,X11,X12,X14,X15,X16,X17,X13,X18,X19,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29
0,0.0,0,7,2005,2017,4404,5,5,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1.0,0,1,2018,2019,174,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1.0,0,7,2012,2018,1999,5,2,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1.0,7,1,2015,2018,1148,4,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1.0,0,7,2017,2018,463,3,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


MODEL SUPPORT VECTOR CLASSIFIER

In [ ]:
clf1 =make_pipeline(StandardScaler(),SVC(gamma="auto"))
clf1.fit(X, y1)
p1 = clf1.predict(t)
clf2 =make_pipeline(StandardScaler(),SVC(gamma="auto"))
clf1.fit(X, y1)
clf2.fit(X, y2)
p2 = clf2.predict(t)

CONVERTING THE FINAL PREDICTIONS INTO A CSV

In [ ]:
ans = {'pet_id':test['pet_id'],'breed_category':p1,'pet_category':p2}
sub = pd.DataFrame(ans)
sub['breed_category'] = sub['breed_category'].astype(int)
sub.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1,2
1,ANSL_76663,0,1
2,ANSL_58259,0,2
3,ANSL_67171,1,1
4,ANSL_72871,0,2


CONVERTING PANDAS DATAFRAME TO CSV

In [ ]:
sub.to_csv('/content/sub.csv',index=False)